In [2]:
import cv2
import numpy as np
import os

# 在圖像上畫出frame和label
def draw_labels(image, labels):
    height, width, _ = image.shape
    for label in labels:
        class_id, x_center, y_center, bbox_width, bbox_height = map(float, label.split())

        # 將 yolo label轉換為frame坐標
        x_center = int(x_center * width)
        y_center = int(y_center * height)
        bbox_width = int(bbox_width * width)
        bbox_height = int(bbox_height * height)

        # 計算frame的左上角坐標
        x1 = int(x_center - bbox_width / 2)
        y1 = int(y_center - bbox_height / 2)
        x2 = int(x_center + bbox_width / 2)
        y2 = int(y_center + bbox_height / 2)

        # 在圖像上畫出frame和label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f'Class {int(class_id)}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image

# 使用鍵盤瀏覽圖像。
# A: 前一張, D: 下一張, EXC退出
def visualize_dataset(image_dir, label_dir):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
    label_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])

    index = 0
    total_images = len(image_files)

    while True:
        image_file = image_files[index]
        label_file = image_file.rsplit('.', 1)[0] + '.txt'

        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, label_file)

        # 加載圖像和label
        image = cv2.imread(image_path)
        try:
            with open(label_path, 'r') as f:
                labels = f.readlines()
        except FileNotFoundError:
            print(f"未找到 {image_file} 的標籤文件。顯示無標籤的圖像。")
            labels = []

        # 在圖像上標出label
        labeled_image = draw_labels(image.copy(), labels)

        # 顯示圖像
        cv2.imshow('Labeled Image', labeled_image)

        key = cv2.waitKey(0) & 0xFF

        if key == 27:  # ESC 鍵退出
            break
        elif key == ord('a') or key == 81:  # A鍵顯示上一張圖像
            index = (index - 1) % total_images
        elif key == ord('d') or key == 83:  # D鍵顯示下一張圖像
            index = (index + 1) % total_images

    cv2.destroyAllWindows()

image_dir = './DataSynthetics/output/images'
label_dir = './DataSynthetics/output/labels_fine'
visualize_dataset(image_dir, label_dir)
